In [8]:
import regionmask
import xarray as xr
import numpy as np
import pandas as pd
import warnings
import cftime
warnings.filterwarnings("ignore")

In [11]:
# Reemplaza 'ruta/a/tu/archivo.nc' con la ruta real a tu archivo NetCdata
data = xr.open_dataset("Datos\states.nc",decode_times=False)

print(data)
data.close()

<xarray.Dataset> Size: 68GB
Dimensions:     (time: 1166, lat: 720, lon: 1440, bounds: 2)
Coordinates:
  * time        (time) float64 9kB 0.0 1.0 2.0 ... 1.163e+03 1.164e+03 1.165e+03
  * lat         (lat) float64 6kB 89.88 89.62 89.38 ... -89.38 -89.62 -89.88
  * lon         (lon) float64 12kB -179.9 -179.6 -179.4 ... 179.4 179.6 179.9
Dimensions without coordinates: bounds
Data variables: (12/16)
    primf       (time, lat, lon) float32 5GB ...
    primn       (time, lat, lon) float32 5GB ...
    secdf       (time, lat, lon) float32 5GB ...
    secdn       (time, lat, lon) float32 5GB ...
    urban       (time, lat, lon) float32 5GB ...
    c3ann       (time, lat, lon) float32 5GB ...
    ...          ...
    pastr       (time, lat, lon) float32 5GB ...
    range       (time, lat, lon) float32 5GB ...
    secmb       (time, lat, lon) float32 5GB ...
    secma       (time, lat, lon) float32 5GB ...
    lat_bounds  (lat, bounds) float64 12kB ...
    lon_bounds  (lon, bounds) float64 23k

In [12]:
data

<xarray.Dataset> Size: 68GB
Dimensions:     (time: 1166, lat: 720, lon: 1440, bounds: 2)
Coordinates:
  * time        (time) float64 9kB 0.0 1.0 2.0 ... 1.163e+03 1.164e+03 1.165e+03
  * lat         (lat) float64 6kB 89.88 89.62 89.38 ... -89.38 -89.62 -89.88
  * lon         (lon) float64 12kB -179.9 -179.6 -179.4 ... 179.4 179.6 179.9
Dimensions without coordinates: bounds
Data variables: (12/16)
    primf       (time, lat, lon) float32 5GB ...
    primn       (time, lat, lon) float32 5GB ...
    secdf       (time, lat, lon) float32 5GB ...
    secdn       (time, lat, lon) float32 5GB ...
    urban       (time, lat, lon) float32 5GB ...
    c3ann       (time, lat, lon) float32 5GB ...
    ...          ...
    pastr       (time, lat, lon) float32 5GB ...
    range       (time, lat, lon) float32 5GB ...
    secmb       (time, lat, lon) float32 5GB ...
    secma       (time, lat, lon) float32 5GB ...
    lat_bounds  (lat, bounds) float64 12kB ...
    lon_bounds  (lon, bounds) float64 23kB ...
Attributes: (12/17)
    host:              UMD College Park
    comment:           LUH2
    contact:           gchurtt@umd.edu, lchini@umd.edu, steve.frolking@unh.ed...
    creation_date:     2016-10-12T18:26:36Z
    title:             Land-Use Harmonization Data Sets
    activity_id:       input4MIPs
    ...                ...
    further_info_url:  http://luh.umd.edu
    frequency:         yr
    institution_id:    UofMD
    institution:       University of Maryland College Park
    realm:             land
    references:        Hurtt, Chini et al. 2011

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Suponiendo que tienes un DataFrame llamado data con columnas 'lon', 'lat' para longitud y latitud
# data = pd.DataFrame({'lon': [-99.1332, -0.1257], 'lat': [19.4326, 51.5074], 'time': ['2021-01-01', '2021-01-02']})
# Define las regiones del AR6 utilizando regionmask
ar6_all = regionmask.defined_regions.ar6.all
# Crear una figura y un subplot con proyección
fig, ax = plt.subplots(figsize=(20, 10), subplot_kw={'projection': ccrs.PlateCarree()})
ax.set_global()

# Añadir características al mapa
ax.add_feature(cfeature.LAND)
#ax.add_feature(cfeature.OCEANS)
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle=':')

# Añadir los puntos de datos al mapa
scatter =ax.scatter(ds['lon'],ds['lat'],  c=climatology[-1], cmap='jet', s=10, transform=ccrs.Geodetic(), label='Ubicaciones')
cbar = plt.colorbar(scatter, ax=ax, orientation='horizontal', pad=0.05)
cbar.set_label('Temperatura (°C)')

ar6_all.plot(ax=ax, add_label=False)
# Añadir leyenda y título
ax.legend()
ax.set_title('Mapa de Ubicaciones')

plt.show()


In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import cartopy.crs as ccrs
import cartopy.feature as cfeature


# Supongamos que 'time' es la primera dimensión y 'space' la segunda
# Primero, selecciona un solo paso de tiempo (por ejemplo, el primero)
single_time_slice = data['primf'].isel(time=0)

# Asegúrate de que 'single_time_slice' sea ahora un array 1D con la misma longitud que los arrays 'lon' y 'lat'
# Es posible que necesites aplanar 'single_time_slice' si aún es 2D después de seleccionar el tiempo
flat_temp_data = single_time_slice.values.flatten()

# Asegúrate de que 'lon' y 'lat' también sean arrays 1D con la misma longitud que 'flat_temp_data'
# Puede que necesites usar ravel() o flatten() si 'lon' y 'lat' no son 1D
flat_lon = data['lon'].values.ravel()
flat_lat = data['lat'].values.ravel()

# Verificar que todos los arrays tienen la misma longitud antes de la interpolación
#assert len(flat_lon) == len(flat_lat) == len(flat_temp_data)

# Ahora puedes hacer la interpolación con 'griddata'
grid_temp = griddata(
    (flat_lon, flat_lat),  # Coordenadas de los puntos
    flat_temp_data,        # Valores de temperatura para un momento en el tiempo
    (lon, lat),            # Malla de coordenadas para la interpolación
    method='cubic'         # Método de interpolación
)

# Crear figura y eje con proyección de Cartopy
fig, ax = plt.subplots(figsize=(20, 10), subplot_kw={'projection': ccrs.PlateCarree()})
ax.set_extent([data['lon'].min() - 5, data['lon'].max() + 5, data['lat'].min() - 5, data['lat'].max() + 5], crs=ccrs.PlateCarree())

# Añadir características del mapa
ax.add_feature(cfeature.LAND, edgecolor='black')
ax.add_feature(cfeature.COASTLINE)

# Visualizar la malla de temperatura
contour = ax.contourf(lon, lat, grid_temp, levels=100, cmap='RdYlBu_r', transform=ccrs.PlateCarree())

# Añadir barra de color
cbar = plt.colorbar(contour, ax=ax, orientation='horizontal', pad=0.05, aspect=50)
cbar.set_label('Temperatura (°C)')

# Añadir título
ax.set_title('Mapa de Temperatura')

plt.show()


ValueError: shape mismatch: objects cannot be broadcast to a single shape.  Mismatch is between arg 0 with shape (1440,) and arg 1 with shape (720,).